# [프로젝트1] 시카고 샌드위치 맛집 분석

웹 크롤링을 활용해 시카고 샌드위치 맛집 분석을 해보려 합니다.
시카고 샌드위치 맛집 분석에서는 샌드위치 맛집의 이름, 샌드위치 이름, 가게의 주소 등의 정보 가져오기를 해 볼 겁니다. 더불어 웹 크롤링을 하여 정보를 가져올 때에는 꽤 시간이 걸리기 때문에 걸리는 시간을 시각화 해 알려주는 tqdm 모듈도 사용해 볼겁니다. 정말 재미있는 부분이라 이번 주차도 신나게 공부할 수 있을 것 같습니다!🐶 그럼 이번 주도 공부 열심히 해봅시다.

### urllib과 BeautifulSoup으로 사이트에 접근하기


지난 시간에 배운 urllib과 BeautifulSoup으로 시카고 샌드위치 맛집 랭킹을 알려주는 사이트를 불러와 볼겁니다. 여기에서는 가게이름, 가게 메인 메뉴, 각 가게 소개 페이지를 가져와 볼거에요. 우선 필요한 모듈들을 모두 import 해봅시다.

In [ ]:
import urllib.request # 403 Forbidden 오류해결
from urllib.request import urlopen
from bs4 import BeautifulSoup

* [시카고 샌드위치 맛집 사이트](https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/)
교재에서는 url을 두 부분으로 나누어 놓았어요. 후에 상대주소를 절대주소 바꾸기 위해서 이렇게 하는 것이 필요해요. (자세한 설명은 후에 상대주소/절대주소를 나눌 때 설명할게요.) 이렇게 하고 html을 읽어와 보기로 해요.

In [ ]:
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
headers = {'User-Agent': 'Mozilla/5.0'}
res = urllib.request.Request(url, headers =headers)
html = urlopen(res).read()
soup = BeautifulSoup(html, "html.parser")
# soup

잘 불러와졌는지 확인해보는 것을 잊지 말구요. 그 다음 크롬의 개발자 도구(F12)를 확인해 우선 1위의 BLT와 다른 정보들이 담겨있는 태그와 클라스가 무엇인지 확인해봅니다. div 태그에 class는 sammy네요. find_all을 사용해 가져와볼게요.

![](img/week5_img_classtag.png)

개발자 도구를 연 후, 1위인 BLT 부분을 선택하면 해당 부분의 태그와 클라스가 뜬 걸 확인할 수 있네요!

In [ ]:
lists = soup.find_all('div', class_='sammy')
# lists

잘 불러와졌네요! 1) 샌드위치 이름 2) 샌드위치 가게 이름 3) 사이트 주소 4) 랭킹까지 모두 들어있죠? 그런데 너무 다양하게 많이 적혀져 있으니, 이름만 우선 get_text()로 모두 가져와 볼게요. for문으로 이를 출력해보겠습니다. len()을 사용해 정확히 50개가 맞는지도 살펴보구요.

In [ ]:
for list in lists[:5]: 
    print(list.get_text())


1
BLT
Old Oak Tap
Read more 


2
Fried Bologna
Au Cheval
Read more 


3
Woodland Mushroom
Xoco
Read more 


4
Roast Beef
Al’s Deli
Read more 


5
PB&L
Publican Quality Meats
Read more 



In [ ]:
len(lists)

50

lists의 길이가 50개가 맞으니 잘 뽑아낸 것 같습니다.